[Java线程池实现原理及其在美团业务中的实践](https://tech.meituan.com/2020/04/02/java-pooling-pratice-in-meituan.html)

内容详情见上面的链接，本文为读后思考，以及一些补充内容。

## 池化（Pooling）
**1.池化的精髓是什么？**  
统一管理（最小化风险），预先申请（最大化收益）。

**2.常见的池化操作？**  
内存池（Memory Pooling）：预先申请内存，提升申请内存速度，减少内存碎片。  
连接池（Connection Pooling）：预先申请数据库连接，提升申请连接的速度，降低系统的开销。  
线程池（Thread Pooling）：将线程的创建、销毁、调度统一管理，提高了计算机的整体性能，保证了对内核的充分利用。  
实例池（Object Pooling）：循环使用对象，减少资源在初始化和释放时的昂贵消耗。

## 内存池（Memory Pooling）
### 内存预分配
为啥写算法题直接把内存拉满？？直接一次性分配一大块，快呀！平时使用数据结构的时候也要预分配内存，特别HashMap，避免导致频繁resize。
#### HashMap初始化大小计算
设需要容量为 $k$   
```
int initCapacity = (int)(k / loadfactor) + 1;
```
其中loadfactor为加载因子，默认为0.75。  
指定初始化大小之后，需要计算最小二次幂作为其容量，该部分由HashMap内部计算，如下
```
static final int tableSizeFor(int cap){
    int n = cap - 1;
    n |= n >>> 1;
    n |= n >>> 2;
    n |= n >>> 4;
    n |= n >>> 8;
    n |= n >>> 16;
    return (n < 0) ? 1 : (n >= MAXIMUN_CAPACITY) ? MAXIMUN_CAPACITY : n + 1;
}
```
或者使用
```
Map<K, V> map = Maps.newHashMapWithExpectedSize(initCapacity);
```
### 手动申请/释放内存存在的问题
new/delete,malloc/free等操作是在堆上进行。  
**更新过程**：操作系统首先会查找***空闲块表*** 找到合适大小的***内存空闲块***（段页式、分配算法：首先适配算法、最佳适配算法、最坏适配算法），若该块内存过大，则需要切割成已分配的部分和较小的***空闲块***，然后系统***更新空闲块表***。  
**释放过程**：系统把释放的内存块重新***加入到空闲内存块表中***，若有可能的话，会合并相邻的空闲块。  

### 内存分配算法
| 算法名称 | 空闲区 | 算法 | 特点 |
| ------ | ------ | ------ | ------ |
| 首先适配算法 | 按**地址**由低到高排列 | 查空闲块表，找到第一个不小于请求的空块，将其**分割**并分配（**产生了碎片**） | 简单、快速分配 |
| 最佳适配算法 | 按**大小**由小到大排列 | 在空闲块表中找到一个第一个不小于请求的最小空块进行分配（**分割后，碎片较小**） | 用最小空间满足需求 |
| 最坏适配算法 | 按**大小**由大到小排列 | 在空闲块中找到一个第一个不小于请求的最大空块进行分配（**分割后空闲块仍然很大**） | 分割后空闲块仍然很大 |

### 内存池优化点
1) 针对特殊情况，例如需要频繁分配释放固定大小的内存对象时，不需要复杂的分配算法和多线程保护。也不需要维护内存空闲表的额外开销，从而获得较高的性能。  
2) 由于开辟一定数量的连续内存空间作为内存池块，因而一定程度上提高了程序局部性，提升了程序性能。  
3) 比较容易控制页边界对齐和内存字节对齐，没有内存碎片的问题。(现代操作系统通常采用段页式)  